# Lab 1: Building a Synchronous Marketing Agent for UniTok

## Introduction
![architecture](./lab1.png)
Welcome to the first lab in our workshop on building asynchronous AI agents with Strands Agents SDK! In this lab, we'll create a synchronous marketing agent for UniTok, our fictional unicorn rental social media platform.

By the end of this lab, you'll understand the basic structure of a Strands Agent, how to configure it with custom tools, and how to use it to generate engaging marketing content.

Let's get started!

## Setup

⚠️⚠️ Please run the [prerequisites](../lab_0/prerequisites.ipynb) before continuing with this lab, if you haven't done so already. ⚠️⚠️

This lab assumes that an ENVIRONMENT VARIABLE `PUBLISH_API_ENDPOINT` is set with the URL to use to publish to UniTok website.

In [ ]:
# Install required packages
%pip install --upgrade strands-agents strands-agents-tools requests boto3

In [ ]:
%env AWS_DEFAULT_REGION=us-west-2 

In [ ]:
# Import required libraries
import json
import os
import requests
from strands import Agent, tool
from strands.models import BedrockModel
import boto3

response = boto3.client('cloudformation').describe_stacks(StackName="UniTokStack")
        
# Get the Unito API endpoint URL
for output in response['Stacks'][0]['Outputs']:
    if output['OutputKey'] == "ApiEndpoint":
        PUBLISH_API_ENDPOINT = f'{output["OutputValue"]}posts'
    if output['OutputKey'] == "DistributionDomainName":
        UNITOK_URL = f'http://{output["OutputValue"]}'
print(f"Using following API endpoint for publishing UniTok posts: {PUBLISH_API_ENDPOINT}")
print(f"You can reach the UniTok site at : {UNITOK_URL}")

## Understanding the UniTok Platform

UniTok is a fictional social media platform where users can browse, rent, and share experiences with unicorns. Our marketing agent will help create engaging posts to promote Unicorn Rentals' services on this platform.

The UniTok backend API is already deployed and running - you don't need to worry about setting up these resources as they're outside the scope of this workshop.

Let's explore what makes a good UniTok post:

> **Effective UniTok Posts:**
> - Are family-friendly and positive
> - Highlight the magical experience of spending time with unicorns
> - Mention the color selection feature when appropriate (colors: pink, blue, purple, green, yellow, and rainbow)
> - Use emojis sparingly but effectively
> - Keep content between 50-200 characters for optimal engagement
> - Appeal to families with children, fantasy enthusiasts, and event planners

## Creating the Agent's Tools

Our agent needs tools to interact with the UniTok platform. Let's start by implementing the `publish_post` tool.

In [ ]:
# TODO: Implement the publish_post tool
@tool
def publish_post(
    content: str, 
    author: str = "Unicorn Rentals", 
    unicorn_color: str = "rainbow", 
    image_url: str = None) -> str:
    """
    Publish a post to the UniTok social media platform.
    
    Args:
        content (str): The text content of the post.
        author (str, optional): The author of the post. Defaults to "Unicorn Rentals".
        unicorn_color (str, optional): The color of the unicorn. Choose from: pink, blue, purple, green, yellow, or rainbow. Defaults to "rainbow".
        image_url (str, optional): URL to an image to include with the post. Defaults to None.
        
    Returns:
        str: A message indicating the post was published successfully, or an error message.
    """
    
    # For this lab, we'll simulate posting to UniTok
    print(f"Publishing post to UniTok: {content}")
    print(f"Author: {author}")
    print(f"Unicorn Color: {unicorn_color}")
    if image_url:
        print(f"Image URL: {image_url}")
    post_data = {
        "content": content,
        "author": author,
        "unicornColor": unicorn_color
    }
    if image_url:
        post_data["imageUrl"] = image_url
    try:
        # Send the post to the API
        response = requests.post(PUBLISH_API_ENDPOINT, json=post_data)
        
        # Check if the request was successful
        if response.status_code == 201:
            post_id = response.json().get("postId")
            return f"Post published successfully! Post ID: {post_id}"
        else:
            return f"Failed to publish post. Status code: {response.status_code}, Response: {response.text}"
    except:
        return "Error publishing post"

## Configuring the Post Generator Agent

Now, let's configure our marketing post generator agent with appropriate instructions and tools.

In [ ]:
# Define the agent's instructions
INSTRUCTIONS = """
You are a creative social media manager for Unicorn Rentals, a company that offers unicorns for rent that kids and grown-ups can play with.

Your task is to create engaging social media posts for UniTok, our unicorn-themed social media platform. 

Important information about Unicorn Rentals:
- We offer unicorns in various colors: pink, blue, purple, green, yellow, and rainbow (our most popular)
- Our new product feature allows customers to pick their favorite color unicorn to rent (default: rainbow)
- Our target audience includes families with children, fantasy enthusiasts, and event planners
- Our brand voice is magical, playful, and family-friendly

When creating posts:
- Keep content family-friendly and positive
- Highlight the magical experience of spending time with unicorns
- Mention the new color selection feature when appropriate
- Use emojis sparingly but effectively
- Keep posts between 50-200 characters for optimal engagement

After generating the post, publish it directly to UniTok using the publish_post tool.
"""

In [ ]:
# Initialize the model
model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0"
    # Add any additional model configuration here
)

# Create the agent
post_generator_agent = Agent(
    model=model,
    system_prompt=INSTRUCTIONS,
    tools=[publish_post],
)

## Testing the Agent

Let's test our agent with various marketing scenarios to see how it performs.

### Scenario 1: General Marketing Post

In [ ]:
response = post_generator_agent("Create a post promoting our unicorn rental service.")
print(response)

### Scenario 2: Seasonal Promotion


In [ ]:
response = post_generator_agent("Create a summer-themed post highlighting our rainbow unicorns.")
print(response)

### Scenario 3: New Feature Announcement

In [ ]:
response = post_generator_agent("Create a post announcing our feature to rent unicorns for next children summer camp in the mountains.")
print(response)

## Examining the Agent's Messages

Let's look at the messages exchanged between us and the agent:

In [ ]:
for message in post_generator_agent.messages:
    print(f"Role: {message['role']}")
    print(f"Content: {message['content']}")
    print("---")

## Analyzing Tool Usage

Let's examine how the agent used our custom tool:

In [ ]:
for message in post_generator_agent.messages:
    for content_item in message.get('content', []):
        if isinstance(content_item, dict) and 'toolUse' in content_item:
            tool_use = content_item['toolUse']
            print(f"Tool: {tool_use['name']}")
            print(f"Input: {tool_use['input']}")
            print("---")

In [ ]:
print(f"You can reach the UniTok site at : {UNITOK_URL}")

## Viewing Posts on the UniTok Website

Now that we've created and published posts using our agent, let's see them on the UniTok website!

To view your posts:
1. Use the **UniTokUrl** displayed above. It can also be found in your CloudFormation deployment output from prerequisites. It should look something like: `https://d123abc456def.cloudfront.net`
2. Open this URL in your web browser
3. You should see the posts that our agent has created and published, displayed in reverse chronological order

Each post shows:
- The content of the post
- The author (which we set to "Unicorn Rentals")
- The unicorn color (visualized with the appropriate color)
- The timestamp when the post was created
- The number of likes (starting at 0)

This demonstrates the end-to-end flow of our agent: it generates creative content based on our prompts, publishes it to the UniTok API, and then we can see the posts on the UniTok website.

## Limitations of Synchronous Agents

Our current implementation has several limitations:

1. **Execution Time Constraints**: The agent must complete its entire task within the execution time limit of the environment.
2. **No Persistence**: If the execution is interrupted, all progress is lost.
3. **Blocking Execution**: The client must wait for the entire process to complete before receiving a response.
4. **Limited Complexity**: Complex workflows requiring external input or long-running processes are difficult to implement.

In the next lab, we'll address these limitations by converting our agent to an asynchronous architecture using AWS Lambda and SQS.

## Conclusion

Congratulations! You've successfully built a synchronous marketing agent for UniTok using the Strands Agents SDK. This agent can generate engaging social media posts based on specific prompts and publish them to the platform.

Key takeaways from this lab:
- How to configure a Strands Agent with custom instructions
- How to implement and use tools with the agent
- How to test the agent with various scenarios

In Lab 2, we'll transform this synchronous agent into an asynchronous one that can handle tasks more efficiently and reliably.